In [4]:
#############################################
#### Exploratory Analysis of FirenzeCard data
#############################################
import sys
sys.path.append('../src/')
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline  
import psycopg2
from features.firenzecard import *
#from utils.dbutils import *

# establish connection to db
#connection = connect(host='', port=5432)

df = pd.read_csv('../src/output/firenzedata_feature_extracted.csv')

In [ ]:
########################
# Card Usage Behaviour
########################

# How many cards are there?
print('How many Firenzecards are there?', len(df['user_id'].unique()))

In [3]:
# How many cards were activated? 
len(df[(df['adults_first_use'] == 1)])

# What is the most common day of activation?
day_of_activation, plot_url_activation = plot_day_of_activation(df, plotname='DOA')
plot_url_activation

In [4]:
# How many users use the card for 24h or less? (not cumulative calculation)
print(len(df[df['total_duration_card_use'] <= 24].user_id.unique()))

# ... 24 - 48h?
print(len(df[(df['total_duration_card_use'] > 24) & (df['total_duration_card_use'] <= 48)].user_id.unique()))

# ... 48 - 72h?
print(len(df[(df['total_duration_card_use'] > 48) & (df['total_duration_card_use'] <= 72)].user_id.unique()))

9162
17929
23939


In [5]:
# How many museums visited per card / per day
total_museums_per_card, plot_url1 = plot_museums_visited_per_card(df, plotname1 = 'Number-museums-per-card')
plot_url1

In [6]:
########################
# Popular Museums
########################

# What are the most popular museums?
popular_museums, plot_url2 = plot_museum_aggregate_entries(df, plotname='PM')
plot_url2

In [7]:
########################
# State Museum Visits
########################

national_museum_entries, plot_url3 = plot_national_museum_entries(connection, export_to_csv=True,
                                                                      export_path='../src/output/')
plot_url3

In [8]:
# How many cards are entering museums with minors? What proportion of all cards is this?
minors = df[df['is_card_with_minors'] == 1]
minors = minors.groupby('user_id').size().to_frame()                         
len(minors)

4914

In [5]:
##############################
# Entries in Museums over time
##############################

museum_list = ['Santa Croce', 'Opera del Duomo', 'Uffizi', 'Accademia',
       'M. Casa Dante', 'M. Palazzo Vecchio', 'M. Galileo', 'M. Bargello',
       'San Lorenzo', 'M. Archeologico', 'Pitti', 'Cappelle Medicee',
       'M. Santa Maria Novella', 'M. San Marco', 'Laurenziana',
       'M. Innocenti', 'Palazzo Strozzi', 'Palazzo Medici',
       'Torre di Palazzo Vecchio', 'Brancacci', 'M. Opificio',
       'La Specola', 'Orto Botanico', 'V. Bardini', 'M. Stefano Bardini',
       'M. Antropologia', 'M. Ebraico', 'M. Marini', 'Casa Buonarroti',
       'M. Horne', 'M. Ferragamo', 'M. Novecento', 'M. Palazzo Davanzati',
       'M. Geologia', 'M. Civici Fiesole', 'M. Stibbert', 'M. Mineralogia',
       'M. Preistoria', 'M. Calcio', 'Primo Conti','All Museums']

In [2]:
df_date, plot_urls = get_museum_entries_per_timedelta_and_plot(df, museum_list, timedelta='date',
                                                          start_date='2016-06-01',
                                                          end_date='2016-09-30', plot=False, export_to_csv=False,
                                                          export_path='../src/output/')

df2_date = df_date['All Museums']
df_date['All Museums'].head()

,date,museum_id,total_entries
0,2016-06-01,1,172
1,2016-06-01,2,134
2,2016-06-01,3,333
3,2016-06-01,4,71
4,2016-06-01,5,55


In [6]:
df_hour, plot_urls = get_museum_entries_per_timedelta_and_plot(df, museum_list, timedelta='hour',
                                                          start_date='2016-06-01',
                                                          end_date='2016-09-30', plot=False, export_to_csv=False,
                                                          export_path='../src/output/')

df2_hour = df_hour['All Museums']
df_hour['All Museums'].head()

,hour,museum_id,total_entries
0,0,<built-in function id>,0.0
1,1,<built-in function id>,0.0
2,2,<built-in function id>,0.0
3,3,<built-in function id>,0.0
4,4,<built-in function id>,0.0


In [10]:
df_dow, plot_urls = get_museum_entries_per_timedelta_and_plot(df, museum_list, timedelta='day_of_week',
                                                          start_date='2016-06-01',
                                                          end_date='2016-09-30', plot=False, export_to_csv=False,
                                                          export_path='../src/output/')

df2_dow = df_dow['All Museums']
df_dow['All Museums'].head()

,day_of_week,museum_id,total_entries
0,0,1,4008.0
1,0,2,3481.0
2,0,3,12261.0
3,0,4,1608.0
4,0,5,699.0


In [12]:
# Timeline of usage(per avg hour, calendar day, calendar month, weekday) - segment per museum
mean_entries_hour, mean_entries_dow, mean_entries_date = get_timelines_of_usage(df2_hour, df2_date, df2_dow)
# mean_entries_hour.head(), mean_entries_dow.head(), mean_entries_date.head()

In [13]:
# Daily Museums entries 
date, date_url = plot_timeseries_button_plot(df_date, timedelta= 'date', plotname='timeseries')
date_url

In [14]:
# Hourly Museums entries 
hour, hour_url = plot_timeseries_button_plot(df_hour, timedelta= 'hour', plotname='timeseries')
hour_url

In [12]:
# Day of Week museum entries
dow, dow_url = plot_timeseries_button_plot(df_dow, timedelta= 'day_of_week', plotname='testing')
dow_url

In [4]:
##########################
## Geographic Timseries map
##########################

# Which museums are full, and which are rather empty, at different times of the day?
# Are they located next to each other?
data, geomap_plot_url = plot_geomap_timeseries(df, df2_hour, date_to_plot='2016-07-10', 
                           plotname='map-test', mapbox_access_token='pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw', min_timedelta=7,
                           max_timedelta=23)
geomap_plot_url

In [20]:
######################
# Museum timeseries correlations
######################

lst = list(df.museum_id.unique())

corr_matrix, high_corr, inverse_corr = get_correlation_matrix(df=df2_hour, lst = lst, corr_method = 'spearman',
                                                              timedelta='hour', timedelta_subset = False, 
                                                              timedeltamin = 0, timedeltamax = 3, 
                                                              below_threshold= -0.7, above_threshold=0.7, 
                                                              export_to_csv=True, export_path='../src/output/')

inverse_corr.head()

,inverse_combinations_1,inverse_combinations_2,values
0,1,25,-0.880952
1,2,28,-0.785714
2,2,42,-0.880952
3,3,28,-0.750000
4,5,17,-1.000000


In [7]:
df.head()

,user_id,museum_id,entrances_per_card_per_museum,museum_name,longitude,latitude,short_name,string,entry_time,adults_first_use,...,is_in_museum_30,is_in_museum_31,is_in_museum_32,is_in_museum_33,is_in_museum_34,is_in_museum_35,is_in_museum_36,is_in_museum_37,is_in_museum_38,is_in_museum_39
0,1459702,1,1,Basilica di Santa Croce,11.262598,43.768754,Santa Croce,C,2016-06-23 15:05:00,0,...,0,0,0,0,0,0,0,0,0,0
1,1459702,3,1,Battistero di San Giovanni,11.254966,43.773131,Opera del Duomo,D,2016-06-23 09:43:00,0,...,0,0,0,0,0,0,0,0,0,0
2,1459702,10,1,Galleria degli Uffizi,11.255607,43.768526,Uffizi,U,2016-06-22 10:04:00,1,...,0,0,0,0,0,0,0,0,0,0
3,1459702,11,1,Galleria dell'Accademia di Firenze,11.258516,43.776755,Accademia,A,2016-06-22 15:49:00,0,...,0,0,0,0,0,0,0,0,0,0
4,1459702,15,1,Museo Casa Dante,11.257062,43.771071,M. Casa Dante,3,2016-06-22 14:26:00,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test, plots = geomplot_per_hour(df, df2_hour, timedelta='hour', min_timedelta=7, max_timedelta=23,
                                plotname = 'test',mapbox_access_token='pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw')

KeyboardInterrupt: 

In [8]:
# for museum in museum_list:
#     print(museum)
#     trace = go.Bar(x=df2_hour[museum][timedelta],
#                    y=df2_hour[museum].total_entries,
#                    name=museum,
#                    visible=True,
#                    marker=Marker(
#                        color='#CC171D',  # set bar colors
#                    ))
#     out[museum] = trace
    
# out

dates = df.date.unique()

dataPanda = []
for date in dates:
    trace = go.Bar(x=df_hour['All Museums']['hour'],
                   y=df_hour['All Museums'].total_entries,
                   name=date,
                   visible=True,
                   marker=Marker(
                       color='#CC171D',  # set bar colors
                   ))
    dataPanda.append(trace) 

dataPanda

[{'marker': {'color': '#CC171D'},
  'name': '2016-06-23',
  'type': 'bar',
  'visible': True,
  'x': 0       0
  Name: total_entries, Length: 399, dtype: float64}]

In [15]:
dataPanda[1].name

'2016-06-22'

In [ ]:
for n in len(dataPanda.name):
    
   args = True, False, False, False, False, False, False, False, False, False, False,
                                        False, False, False, False, False, False, False, False, False, False, False,
                                        False, False, False, False, False, False, False, False, False, False, False,
                                        False, False, False, False, False, False, False, False